<a href="https://colab.research.google.com/github/sada1908/githublearnings/blob/main/SL_PythonMod_Assign_01a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Objectives:
# 1. Design and implement a personal expense tracker that enables users to manage their expenses
# 2. Allow users to categorize expenses and set monthly budgets
# 3. Implement file-handling functionality to save and load expense data
# 4. Create an interactive, menu-driven interface for ease of use

In [2]:
from datetime import datetime

In [3]:
import csv

In [4]:
# Populate teh Categories of expenses
category_list = ('Food', 'Travel', 'Entertainment', 'Other')

In [5]:
# Initialize an empty list to act as the expense ledger
expense_ledger = []

In [6]:
# convert this cell code to defien a funciton for adding an expense list
def add_expenses(category_list):
  expense_dict_template = {}
  date_str = input('Enter the date of the expense in the format YYYY-MM-DD: ')
  try:
    expense_dict_template['date'] = datetime.strptime(date_str, '%Y-%m-%d')
  except ValueError:
    print("Invalid date format. Please use YYYY-MM-DD.")
    return None # Return None if date format is invalid

  for cat in category_list:
    while True: # Loop to ensure valid amount input
      try:
        exp_amt = float(input(f'Enter the amount spent on {cat}: '))
        expense_dict_template[cat] = exp_amt  # Assign the amount here
        break # Exit the inner loop if input is valid float
      except ValueError:
        print("Invalid input. Please enter a number for the amount.")

  expense_dict_template['desciption'] = input('Enter a brief description of the expenses :')
  return expense_dict_template # Return the expense dictionary

In [7]:
# define a function to tarck budget for the categories in the given date span
def expense_tracker(expense_ledger, start_date, end_date):
  total_expenses = 0
  for expense in expense_ledger:
    if start_date <= expense['date'] <= end_date:
      for category, amount in expense.items():
        if category != 'date' and category != 'desciption':
          total_expenses += amount
  return total_expenses

In [8]:
def save_expenses_to_csv(expense_ledger, filename="expense_ledger.csv"):
    if not expense_ledger:
        print("No expenses to save.")
        return

    # Get all unique keys from the dictionaries in the ledger
    all_keys = set()
    for expense in expense_ledger:
        all_keys.update(expense.keys())

    # Define the order of headers, ensuring 'date' and 'desciption' are included
    header = ['date'] + [key for key in all_keys if key not in ['date', 'desciption']] + ['desciption']

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)

        writer.writeheader()
        for expense in expense_ledger:
            writer.writerow(expense)

    print(f"Expenses saved successfully to {filename}")

# Example usage (assuming expense_ledger is already populated)
# save_expenses_to_csv(expense_ledger)

In [18]:
# write a menu of options for adding expenses , viewing expenses , tracking expenses, saving expenses to a file , save & exit , reinitialising the expense ledger and changing the category list

def run_expense_tracker_menu():
    global category_list # Declare category_list as global
    global expense_ledger # Declare expense_ledger as global
    print('Welcome to the Expense Tracker!')
    while True:
        print('\nPlease select an option as per the menu below:')
        print('1. Add expense')
        print('2. View expenses')
        print('3. Track budget')
        print('4. Save expenses')
        print('5. Exit')
        print('6. Reinitialise the expense ledger')
        print('7. Change the category list')

        option = input('Enter your option 1-7: ')

        if option == '1':
            while True:
                new_expense = add_expenses(category_list)
                if new_expense is not None: # Check if add_expenses returned a valid dictionary
                    expense_ledger.append(new_expense) # Append the new expense to the ledger
                inp = input('Do you want to add another expense? (y/n): ')
                if inp.lower() != 'y':
                    break
            print(expense_ledger)
            print('Expenses added successfully!')

        elif option == '2':
            print('Expenses:')
            for expense in expense_ledger:
                print(expense)

        elif option == '3':
            print('Provide your budget for a given period')
            budget = float(input('Enter your budget: '))
            print('give the time period for the budget')
            start_date_str = input('Enter the start date in the format YYYY-MM-DD: ')
            end_date_str = input('Enter the end date in the format YYYY-MM-DD: ')
            try:
                start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
                end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
                spent_budget = expense_tracker(expense_ledger, start_date, end_date)
                if spent_budget > budget:
                    print(f'You have exceeded your budget by {spent_budget - budget}')
                else:
                    print(f'You have {budget - spent_budget} left for the month')
            except ValueError:
                print("Invalid date format. Please use YYYY-MM-DD.")


        elif option == '4':
            file_path = '/content/drive/MyDrive/Simplilearn_02_AgenticAICourse/Simplilearn_Python_01'
            # just take the first 3 leters of input and convert to lowercase
            month,year = input('Enter the month and year in the format mmm-yyyy: ').split('-')
            month = month.lower()
            if month not in ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']:
                print('Invalid month. Please enter a valid month.')
                continue
            filename = month + year + '_expenses.csv'
            filename = file_path + '/' + filename
            save_expenses_to_csv(expense_ledger, filename)
            print('Expenses saved successfully!')


        elif option == '5':
            print('Thank you for using the Expense Tracker!')
            break # Exit the main menu loop

        elif option == '6':
            expense_ledger = []
            print('Expense ledger reinitialized!')

        elif option == '7':
            print('Please enter the new category list')
            new_category_list = input('Enter the new category list separated by commas: ')
            new_category_list = new_category_list.split(',')
            new_category_list = [cat.strip() for cat in new_category_list]
            # Ensure the category_list variable is updated in the global scope
            category_list = new_category_list
            print('Category list updated successfully!')

        else:
            print('Invalid option. Please enter a number between 1 and 7.')

# To run the menu, call the function:
# run_expense_tracker_menu()

In [14]:
def load_expenses_from_csv(filename="expense_ledger.csv"):
    global expense_ledger  # Declare expense_ledger as global
    expense_ledger = []  # Reinitialize the ledger before loading

    try:
        with open(filename, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                # Convert date string back to datetime object
                try:
                    row['date'] = datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
                except ValueError:
                    # Handle potential errors in date format if necessary
                    print(f"Skipping row due to invalid date format: {row}")
                    continue

                # Convert expense amounts from string to float
                for key, value in row.items():
                    if key not in ['date', 'desciption']:
                        try:
                            row[key] = float(value)
                        except ValueError:
                            print(f"Skipping value for {key} due to invalid number format: {value}")
                            row[key] = 0.0 # or handle as an error

                expense_ledger.append(row)
        print(f"Expenses loaded successfully from {filename}")
    except FileNotFoundError:
        print(f"No expense file found at {filename}. Starting with an empty ledger.")
    except Exception as e:
        print(f"An error occurred while loading expenses: {e}")

In [19]:
if __name__ == "__main__":
    month_name = input('Enter the month name you want to do expenses from (e.g., Jan, Feb, etc.): ').lower()
    year = input('Enter the year: ')
    file_name = month_name + year + '_expenses.csv'
    path = '/content/drive/MyDrive/Simplilearn_02_AgenticAICourse/Simplilearn_Python_01'
    file_path = path + '/' + file_name
    try:
      load_expenses_from_csv(file_path)
    except FileNotFoundError:
      print(f"No expense file found at {file_path}. Starting with an empty ledger.")
    run_expense_tracker_menu()

Enter the month name you want to do expenses: aug
Enter the year: 2025
Expenses loaded successfully from /content/drive/MyDrive/Simplilearn_02_AgenticAICourse/Simplilearn_Python_01/aug2025_expenses.csv
Welcome to the Expense Tracker!

Please select an option as per the menu below:
1. Add expense
2. View expenses
3. Track budget
4. Save expenses
5. Exit
6. Reinitialise the expense ledger
7. Change the category list
Enter your option 1-7: 2
Expenses:
{'date': datetime.datetime(2025, 8, 1, 0, 0), 'Other': 100.0, 'Travel': 100.0, 'Entertainment': 0.0, 'Food': 500.0, 'desciption': 'abc'}
{'date': datetime.datetime(2025, 8, 5, 0, 0), 'Other': 100.0, 'Travel': 1000.0, 'Entertainment': 0.0, 'Food': 100.0, 'desciption': 'def'}
{'date': datetime.datetime(2025, 8, 10, 0, 0), 'Other': 100.0, 'Travel': 0.0, 'Entertainment': 100.0, 'Food': 200.0, 'desciption': 'xyz'}

Please select an option as per the menu below:
1. Add expense
2. View expenses
3. Track budget
4. Save expenses
5. Exit
6. Reinitiali